In [15]:
!nvidia-smi

Sun Jan 25 18:07:28 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import os

# Get RT-DETR
if not os.path.isdir("/content/RT-DETR"):
  %cd /content
  print("Cloning RT-DETR")
  !git clone https://github.com/lyuwenyu/RT-DETR.git
else:
  print("RT-DETR is already cloned.")

%cd RT-DETR/rtdetrv2_pytorch
!pip install --upgrade pip
!pip install -r requirements.txt
%cd ..

# Get pretrained weights <- starting training from here
if not os.path.isfile("/content/RT-DETR/pretrained/rtdetrv2_r18vd_120e_coco.pth"):
  !mkdir -p pretrained
  !wget -P pretrained https://github.com/lyuwenyu/storage/releases/download/v0.1/rtdetrv2_r18vd_120e_coco.pth

RT-DETR is already cloned.
[Errno 2] No such file or directory: 'RT-DETR/rtdetrv2_pytorch'
/content/RT-DETR/RT-DETR
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
/content/RT-DETR
/content


In [18]:
# Copy and unzip data from GDrive
if not os.path.isfile("/content/soccertrack.zip"):
  print('Copying soccertrack dataset ...')
  !cp /content/drive/MyDrive/datasets/soccertrack.zip .

if not os.path.isdir("soccertrack"):
  print('Unzipping soccertrack dataset ...')
  !unzip -q soccertrack.zip -d soccertrack

Copying soccertrack dataset ...
Unzipping soccertrack dataset ...


In [19]:
import torch
device = torch.cuda.is_available()
print(f"GPU device for training / inference: {device}")

GPU device for training / inference: True
